In [200]:
import copy
import random
import math
# Initialize all the matrices required for the hidden markov models (for testing only)
N = 2
M = 3
T = 4
pi = [0.60, 0.40]
O = [0, 1, 0, 2]
A = [[0.70, 0.30], [0.40, 0.60]]
B = [[0.10, 0.40, 0.50], [0.70, 0.20, 0.10]]

In [217]:
# The Forward algorithm. Alpha-pass
# pi represents probability of initial states
# A is the state transition matrix
# B is the observation probability matrix
# N is the number of distinct states of the markov process
# T is the length of the observation sequence
# M is the number of ovservation symbols
# O is the observation sequence
def alpha_pass(pi, A, B, N, M, T, O):
    # The alpha pass calculates the probability P(O_0, O_1, O_2 ... O_t, X_T = q_i | model)

    # initialize the zeroth alpha-pass
    alpha = []
    scaled_constants = []
    const = 0
    for states in range(0, N):
        alpha.append([pi[states] * B[states][O[0]]]);
        const += alpha[states][0] # for normalization of alpha constants
    
    scaled_constants.append(1 / const)
    
    for states in range(0, N): # Nomalizing
        alpha[states][0] *= scaled_constants[0]
    
    # starting the DP algorithm here
    for time in range(1, T):

        const = 0

        # for each state
        for states in range(0, N):
            # calculate transition probabilities from each state in time-1 and add it
            prob = 0
            for prev_states in range(0, N):
                prob += (alpha[prev_states][time - 1] * A[prev_states][states]);
            
            # multiply it with probabilities to get the observation symbols
            try:
                alpha[states].append(prob * B[states][O[time]])
            except IndexError:
                print(time)
                print(O[time])
            const += alpha[states][time]
        
        # Normalizing
        for states in range(0, N):
            alpha[states][time] /= const
        scaled_constants.append(1 / const)

    return alpha, scaled_constants

In [202]:
alpha, scaled_constants = alpha_pass(pi, A, B, N, M, T, O)
print(alpha)

[[0.17647058823529413, 0.6234817813765182, 0.16880093131548313, 0.8039793968596827], [0.8235294117647058, 0.3765182186234818, 0.8311990686845169, 0.19602060314031738]]


In [203]:
# The backward algorithm. Beta-pass
# pi represents probability of initial states
# A is the state transition matrix
# B is the observation probability matrix
# N is the number of distinct states of the markov process
# T is the length of the observation sequence
# M is the number of ovservation symbols
# O is the observation sequence
def beta_pass(pi, A, B, N, M, T, O, scaled_constants):
    # initialize the beta-pass
    beta = [[scaled_constants[T - 1]] * T for _ in range(0, N)]
    
    # The algorithm starts here
    for time in range(T - 2, -1, -1):
        
        for states in range(0, N):
            # find the sum of transition probabilities multiplied with the observational probability and the beta-pass of next states
            beta[states][time] = 0
            for next_states in range(0, N):
                beta[states][time] += (A[states][next_states] * B[next_states][O[time + 1]] * beta[next_states][time + 1])
            beta[states][time] *= scaled_constants[time]
    
    return beta

In [204]:
beta = beta_pass(pi, A, B, N, M, T, O, scaled_constants)
print(beta)

[[3.1361634958876796, 2.866993436902883, 3.8988119963446044, 3.568164825122539], [2.899393536595498, 4.392290437816732, 2.667608208025256, 3.568164825122539]]


In [205]:
# the procedure to find the digammas and gammas
def digammas(pi, A, B, N, M, T, O, alpha, beta):

    digamma = [[[1] * T for _ in range(0, N)] for _ in range(0, N)]
    gamma = [[1] * T for _ in range(0, N)]

    # P(O | model)
    P_O = 0
    for i in range(0, N):
        P_O += alpha[i][T - 1]
    
    for time in range(0, T - 1):
        for stat1 in range(0, N):
            gamma[stat1][time] = 0
            for stat2 in range(0, N):
                digamma[stat1][stat2][time] = (alpha[stat1][time] * A[stat1][stat2] * B[stat2][O[time + 1]] * beta[stat2][time + 1]) / P_O;
                gamma[stat1][time] += digamma[stat1][stat2][time]
    
    # Special case for y(T - 1)
    for stat in range(0, N):
        gamma[stat][T - 1] = alpha[stat][T - 1]
    
    return digamma, gamma

In [206]:
digamma, gamma = digammas(pi, A, B, N, M, T, O, alpha, beta)
print(digamma, "\n\n")
print(gamma)

[[[0.14166320511755423, 0.1701586774113151, 0.21080834094874137, 1], [0.046506604635706585, 0.34927307468638374, 0.01806928636703498, 1]], [[0.3777685469801446, 0.058718949904461255, 0.5931710559109413, 1], [0.4340616432665947, 0.42184929799784, 0.1779513167732824, 1]]] 


[[0.1881698097532608, 0.5194317520976989, 0.22887762731577635, 0.8039793968596827], [0.8118301902467393, 0.48056824790230124, 0.7711223726842238, 0.19602060314031738]]


In [213]:
# Estimating the model from here on
def estimate(N, M, T, O):
    # initialize the initial probability here
    pi = []
    for i in range(0, N):
        pi.append(abs(random.gauss(1 / N, 0.05)))
    
    su = sum(pi)
    
    for i in range(0, N):
        pi[i] /= su
    
    # initialize the state transition matrix
    A = []
    for i in range(0, N):
        A.append([])
        for j in range(0, N):
            A[i].append(abs(random.gauss(1 / N, 0.05)))
        su = sum(A[i])
        for j in range(0, N):
            A[i][j] /= su
    
    # initialize the symbol emission probability matrix
    B = []
    for i in range(0, N):
        B.append([])
        for j in range(0, M):
            B[i].append(abs(random.gauss(1 / M, 0.05)))
        su = sum(B[i])
        for j in range(0, M):
            B[i][j] /= su
    print(pi)
    print(A)
    print(B, end="\n\n")

    # max_iter is a explicit condition to stop the recursion.
    max_iter = 100
    for it in range(0, max_iter):
        alpha, scaled_constants = alpha_pass(pi, A, B, N, M, T, O)
        beta = beta_pass(pi, A, B, N, M, T, O, scaled_constants)
        digamma, gamma = digammas(pi, A, B, N, M, T, O, alpha, beta)

        logProb = 0
        for time in range(0, T):
            logProb += math.log(scaled_constants[time])
        logProb *= -1

        if (it % 2 == 0):
            print(logProb)

        # Re-estimate the initial probability matrix
        for stat in range(0, N):
            pi[stat] = gamma[stat][0]
        
        # Re-estimate the state transition matrix
        for stat in range(0, N):
            denom = 0
            for time in range(0, T - 1):
                denom += gamma[stat][time]
            for anot in range(0, N):
                numer = 0
                for time in range(0, T - 1):
                    numer += digamma[stat][anot][time]
                A[stat][anot] = numer / denom
        
        # Re-estimate the symbol emission probability matrix
        for stat in range(0, N):
            denom = 0
            for time in range(0, T):
                denom += gamma[stat][time]
            for anot in range(0, M):
                numer = 0
                for time in range(0, T):
                    if (O[time] == anot):
                        numer += gamma[stat][time]
                B[stat][anot] = numer / denom
    
    # return the model
    return pi, A, B

In [208]:
pi, A, B = estimate(N, M, T, O)
print(pi)
print(A)
print(B)

[0.4979850058304866, 0.5020149941695135]
[[0.5558864470476419, 0.4441135529523581], [0.5459949490953088, 0.45400505090469123]]
[[0.4125624183678257, 0.2182537427202242, 0.3691838389119501], [0.28950167786849057, 0.3228081348207471, 0.3876901873107623]]

-4.378769946759489
-4.125098760908314
-3.908146982268852
-2.3822415075236227
-1.3880470094583401
-1.3862943611200298
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.3862943611198906
-1.386294

In [209]:
import re
# War and peace starts here
file = open("war-and-peace.txt", "r", encoding="ascii", errors="ignore")

# Cleaning the data starts here
data = file.read().replace('\n', ' ') # remove new lines
data = data.lower() # make everything lowercase
data = ''.join(ch for ch in data if (ch.isalpha() or ch == ' ')) # remove any characters that are not a-z
data = re.sub(' +', ' ', data) # remove extra spaces
data = re.sub(' +', ' 26 ', data) # change spaces to the number 26
for i in range(0, 26): # change the characters to number corresponding to their labels in the observation sequence. 0 means a, 1 means b and so on upto z
    data = re.sub(chr(ord('a') + i) + '+', ' ' + str(i) + ' ', data)
data = re.sub(' +', ' ', data) # again remove the extra spaces to make the numbers ready to convert

# load the integers in an observation sequence
observation_sequence = []
T = 0
for each_word in data.split(' '):
    if (each_word != '' and T < 50000):
        observation_sequence.append(int(each_word))
        T += 1
    elif(T >= 50000):
        break

file.close()

In [220]:
T = len(observation_sequence)
N = 2
M = 27
pi, A, B = estimate(N, M, T, observation_sequence)
print(pi)
print(A)
print(B)

[0.4990428709408311, 0.500957129059169]
[[0.48329367623395164, 0.5167063237660483], [0.48239457422578236, 0.5176054257742175]]
[[0.10128338536498013, 0.10985979236415785, 0.0040985989004216836, 0.0414264980047653, 0.05131351573791775, 0.003750939794596347, 0.03209083290490865, 0.009616919492324372, 0.005857506723363556, 0.05850584340821094, 0.04809163180186161, 0.03447170487730992, 0.04576369580021104, 0.008095568177909082, 0.0794479559850584, 0.026148477157657064, 0.009371614876541125, 0.046177859090570886, 0.06070921903977031, 0.033035155954073914, 0.06274547282182857, 0.028970351327455027, 0.02237438079477027, 0.0058670693689962675, 0.013651488869754567, 0.02849474530007244, 0.028779776060513036], [0.016579897287304687, 0.023613102097551734, 0.03487551386335784, 0.09029263164679124, 0.004107163682466951, 0.02740955688230119, 0.03995465748545425, 0.027512186623733698, 0.008281075704353507, 0.020747661931900894, 0.007833914078972545, 0.03431739883275322, 0.03932217751253951, 0.0134037